In [1]:
!pip install transformers
!pip install rouge

In [2]:
!pip install rouge.score nltk py7zr

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.7 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=69d0e6d7a3fcc6c6ca32591eb7dad7b465f613aeb20140ce6906a5e729bec9af
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge.score


In [3]:
!pip install transformers datasets 


In [4]:
import numpy as np
import pandas as pd

import nltk
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
nltk.download('punkt')

import re
import string
import csv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from rouge import Rouge
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from datasets import load_dataset, load_metric

In [6]:
df1=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
df2=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv")
df3=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")

In [7]:
df1.drop_duplicates()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [8]:
from datasets import Dataset,DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_pandas(df1),
    "validation": Dataset.from_pandas(df2),
    "test": Dataset.from_pandas(df3)
})

In [9]:
metric = load_metric('rouge')

In [10]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

In [11]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints="sshleifer/pegasus-cnn-ft-v2"

In [12]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [13]:
def preprocess_data(data_to_process):
    inputs = [text for text in data_to_process['article']]
  #tokenize text
    model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize highlights
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(data_to_process['highlights'], max_length=max_target, padding='max_length', truncation=True)
    
    model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
    return model_inputs

In [14]:
tokenize_data = dataset.map(preprocess_data, batched = True, remove_columns=['id', 'article', 'highlights'])

  0%|          | 0/288 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [34]:
del tokenize_data

In [15]:
#sample the data
train_sample = tokenize_data['train'].select(range(5000))
validation_sample = tokenize_data['validation'].select(range(1000))
test_sample = tokenize_data['test'].select(range(500))

In [16]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM

In [17]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [18]:
del df1
del df2
del df3

In [19]:
batch_size = 1
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
#####################
# metrics
# compute rouge for evaluation 
#####################

def compute_rouge(pred):
    predictions, labels = pred
  #decode the predictions
    decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
    res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
    res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

    pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    res['gen_len'] = np.mean(pred_lens)

    return {k: round(v, 4) for k, v in res.items()}
     

In [21]:
args = transformers.Seq2SeqTrainingArguments(
    'news-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True
     

In [22]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [23]:
train_sample

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [24]:
validation_sample

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [25]:
trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=train_sample,
    eval_dataset=validation_sample,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [26]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.727900,0.946639,44.362600,21.621700,31.261600,31.271900,83.950000
2,0.575200,0.909309,44.604900,21.956700,31.693400,31.676600,77.524000
3,0.537300,0.913100,44.573600,21.929600,31.655200,31.651700,79.335000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3750, training_loss=1.4074199055989582, metrics={'train_runtime': 14199.5689, 'train_samples_per_second': 1.056, 'train_steps_per_second': 0.264, 'total_flos': 2.167098310656e+16, 'train_loss': 1.4074199055989582, 'epoch': 3.0})

In [27]:
trainer.save_model("/kaggle/working/pagasis")

In [ ]:
testing_article= """Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.Â 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for space in the overhead lockers, crashing elbows and seat back kicking? Tests conducted by the FAA use planes with a 31 inch pitch, a standard which on some airlines has decreased . Many economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches . Cynthia Corbertt, a human factors researcher with the Federal Aviation Administration, that it conducts tests on how quickly passengers can leave a plane. But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased, reported the Detroit News. The distance between two seats from one point on a seat to the same point on the seat behind it is known as the pitch. While most airlines stick to a pitch of 31 inches or above, some fall below this. While United Airlines has 30 inches of space, Gulf Air economy seats have between 29 and 32 inches, Air Asia offers 29 inches and Spirit Airlines offers just 28 inches. British Airways has a seat pitch of 31 inches, while easyJet has 29 inches, Thomson's short haul seat pitch is 28 inches, and Virgin Atlantic's is 30-31."""

In [ ]:
model_inputs = tokenizer(testing_article,  max_length=max_input, padding='max_length', truncation=True)

In [ ]:
model_inputs

{'input_ids': [0, 27172, 5324, 141, 3286, 3202, 2082, 7, 28, 562, 2735, 8, 2735, 116, 590, 2284, 1530, 9, 82, 602, 7, 5, 12034, 6, 103, 2320, 32, 8026, 114, 519, 215, 6515, 66, 8449, 16, 2057, 3670, 23, 810, 4, 252, 224, 14, 5, 16921, 980, 15, 16482, 25934, 11372, 16, 45, 129, 9800, 111, 24, 18, 2057, 84, 474, 8, 1078, 11, 4854, 4, 901, 87, 9316, 37333, 81, 5, 3124, 1079, 6, 16921, 980, 15, 8449, 2057, 84, 474, 8, 1078, 11, 4854, 116, 152, 186, 6, 10, 121, 4, 104, 2267, 7640, 333, 278, 62, 30, 5, 641, 9, 6586, 26, 23, 10, 285, 1576, 14, 150, 5, 168, 16, 1372, 7, 278, 2820, 13, 3122, 4731, 15, 8449, 6, 24, 630, 75, 14808, 10246, 10, 3527, 1280, 9, 980, 13, 5868, 4, 128, 1121, 10, 232, 147, 3122, 33, 55, 659, 7, 980, 8, 689, 87, 5868, 1598, 26, 6127, 1063, 4306, 102, 6, 2267, 4915, 15, 5, 1540, 4, 2537, 50141, 108, 243, 16, 86, 14, 5, 30510, 8, 21419, 185, 10, 1413, 13, 26899, 1416, 9, 3670, 955, 125, 115, 2180, 154, 15, 8449, 483, 7, 55, 1473, 743, 87, 2190, 13, 980, 11, 5, 15258, 7014,

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])

***** Running Prediction *****

  Num examples = 1

  Batch size = 1


In [ ]:
raw_pred

array([[    2,     0,     0,     0, 42909, 50141,   625, 20946, 50141,
        13839, 50141, 28357, 16921,   980,    15,  8449,    16,  2057,
           84,   474,     8,  1078,    11,  4854,   479, 50118,  3048,
         2267,  7640,   333,    26,    14,   150,     5,   168,    16,
         1372,     7,   278,  2820,    13,  3122,  4731,    15,  8449,
            6,    24,   630,    75, 14808, 10246,    10,  3527,  1280,
            9,   980,    13,  5868,   479, 50118,   565, 10092,  2964,
           30,     5, 21419,   304,  8449,    19,    10,  1105, 10468,
         2418,  3242,     6,    10,  2526,    61,    15,   103,  8537,
           34,  8065,   479, 50118, 20556,  6503,    34,   389,  4877,
            9,   980,     6,   150,  4602,  1754,   866,  3202,    33,
          227,  1132,     8,  2107,  4877,   479,     2,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
      

In [ ]:
testing_summary = tokenizer.decode(raw_pred[0])

In [ ]:
testing_summary

"</s><s><s><s>Consumer\xa0advisory\xa0group\xa0say shrinking space on planes is putting our health and safety in danger.\nUS consumer advisory group said that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.\nTests conducted by the FAA use planes with a 31 inch seat pitch, a standard which on some airlines has decreased.\nUnited Airlines has 30 inches of space, while Gulf Air economy seats have between 29 and 32 inches.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>"

In [ ]:
testing_highlight = """Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer ."""

In [ ]:
rouge = Rouge()
rouge.get_scores(testing_summary, testing_highlight)

[{'rouge-1': {'r': 0.3333333333333333,
   'p': 0.15942028985507245,
   'f': 0.21568627013264136},
  'rouge-2': {'r': 0.12121212121212122,
   'p': 0.04819277108433735,
   'f': 0.06896551317033318},
  'rouge-l': {'r': 0.3333333333333333,
   'p': 0.15942028985507245,
   'f': 0.21568627013264136}}]

In [ ]:
#trying to test all 500 summaries

In [25]:
test_sample

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [28]:
test_df = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')

In [29]:
test_df.columns

Index(['id', 'article', 'highlights'], dtype='object')

In [33]:
del test_df

In [30]:
hyps = []
with open('/kaggle/working/fine_tuned_bart-cnn_12_16_22.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Article', 'Original Summary', 'Model Output'])
    for i in range(len(test_sample)):
        raw_pred, _, _ = trainer.predict([test_sample[i]])
        our_summ= tokenizer.decode(raw_pred[0])
        hyps.append(our_summ)
        writer.writerow([test_df.article[i], test_df.highlights[i], our_summ])

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [31]:
fine_tuned_bart_cnn_12_16_22_results = pd.read_csv('/kaggle/working/fine_tuned_bart-cnn_12_16_22.csv')

In [32]:
rouge = Rouge()
rouge.get_scores(fine_tuned_bart_cnn_12_16_22_results['Original Summary'], fine_tuned_bart_cnn_12_16_22_results['Model Output'], avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.37717154639878614,
  'p': 0.4358553632044702,
  'f': 0.39329813426938115},
 'rouge-2': {'r': 0.17502080912146217,
  'p': 0.2046462537692974,
  'f': 0.182006980356979},
 'rouge-l': {'r': 0.3547686247700814,
  'p': 0.40969315836575954,
  'f': 0.36984299650521985}}

In [ ]:
model_checkpoints1="philschmid/distilbart-cnn-12-6-samsum"

In [ ]:
tokenizer1 = transformers.AutoTokenizer.from_pretrained(model_checkpoints1)

In [ ]:
model1 = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints1)

In [ ]:
batch_size = 1
#collator to create batches. It preprocess data with the given tokenizer
collator1 = transformers.DataCollatorForSeq2Seq(tokenizer1, model=model1)

In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'news-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )

In [ ]:
trainer = Seq2SeqTrainer(
    model1, 
    args,
    train_dataset=train_sample,
    eval_dataset=validation_sample,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("/kaggle/working/my_fine_tuned_model1")